In [97]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, Image, display, HTML
import msgpack as mp
import msgpack_numpy as mpn
import base64
from datetime import datetime
import os
from tqdm import tqdm
from datetime import timedelta
import keyboard
import time

In [98]:
# base_pth = "C:\mira\splitVideos\SAMXXXXXXXXU010120000000XXXXXXXXX\Session 08-02-22_13-24-38_725"
# base_pth = r"C:\mira\splitVideos\SAMXXXXXXXXU010120000000XXXXXXXXX\Session 04-02-22_14-03-00_359"
# base_pth = r"C:\mira\splitVideos\SAMXXXXXXXXU010120000000XXXXXXXXX\Session 10-02-22_09-14-15_436"
# base_pth = r"C:\mira\splitVideos\IMUXXXXHC05U010120000000XXXXXXXXX\Session 17-02-22_13-45-40_738" # first exp on hc 05
# base_pth = r"C:\mira\splitVideos\IMUXXXXHC05U010120000000XXXXXXXXX\Session 17-02-22_17-13-29_526" # second exp on hc05
# base_pth = r"C:\mira\splitVideos\IMUXXXXHC05U010120000000XXXXXXXXX\Session 18-02-22_11-02-02_575" # third with datetime corrected exp on hc05
# base_pth = r"C:\mira\splitVideos\IMUXXXXHC05U010120000000XXXXXXXXX\Session 18-02-22_12-12-01_66" # fourth
base_pth = r"C:\mira\splitVideos\IMUXXXXHC05U010120000000XXXXXXXXX\Session 18-02-22_13-03-19_712" # fifth

_prm = os.listdir(base_pth)

for i in _prm:
    if i.startswith("PARAMS"):
        _p_fname = i
prm_file = base_pth + "\\" + _p_fname
prm_file


p = open(prm_file, "rb")
unpacker = mp.Unpacker(p, object_hook=mpn.decode)
prm = []
for unpacked in unpacker:
    prm.append(unpacked)
prm.pop(0)
prm.pop(0)


video_time = []
for i in prm:
    video_time.append(datetime.strptime(i[0], '%Y-%m-%d %H:%M:%S.%f'))
v_start = video_time[0]
v_start


"""for df"""
video_time_df = []
for i in prm:
    video_time_df.append(i[0])



In [99]:
video_data = cv2.VideoCapture(base_pth+"//Video.avi")

_i = 0
frames = []
while video_data.isOpened:
    
    ret, frame = video_data.read()
    if not ret:
        break
    frames.append(frame)
    frame = cv2.flip(frame, 1)
    vid_diff = video_time[_i] - video_time[0] 
    _vid_diff = vid_diff.total_seconds()
    _i += 1
    # font
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    # org
    org = (50, 50)
    
    # fontScale
    fontScale = 1
    
    # Blue color in BGR
    color = (255, 0, 0)
    
    # Line thickness of 2 px
    thickness = 2
    
    # Using cv2.putText() method
    image = cv2.putText(frame, str(video_time[_i])[10:], org, font, 
                    fontScale, color, thickness, cv2.LINE_AA)
    if ret:
        cv2.imshow("",frame)
        # plt.show()
        
    if cv2.waitKey(25) & 0xFF == ord('q'):
          break
    
    
    time.sleep(0.001)
    
cv2.destroyAllWindows()
        

In [124]:
video_data = cv2.VideoCapture(base_pth+"//Video.avi")
video_data

<VideoCapture 0000027390752EF0>

In [127]:

"""frame by frame"""
video_data = cv2.VideoCapture(base_pth+"//Video.avi")

frame_no = 10
while video_data.isOpened:
    ret = True
    video_data.set(1, frame_no)

    if video_data:
        frame = video_data.read()

    # print(frame)
    
    if not ret:
        break
    if keyboard.is_pressed("a"):
        frame_no+=1
        print(frame_no)
    
    if keyboard.is_pressed("d"):
        frame_no-=1
    frame = cv2.flip(frame, 1)

    _i += 1
    # font
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    # org
    org = (50, 50)
    
    # fontScale
    fontScale = 1
    
    # Blue color in BGR
    color = (255, 0, 0)
    
    # Line thickness of 2 px
    thickness = 2
    
    # Using cv2.putText() method
    image = cv2.putText(frame, str(video_time[frame_no])[10:], org, font, 
                    fontScale, color, thickness, cv2.LINE_AA)
    if ret:
        cv2.imshow("",frame)
        # plt.show()
        
    if cv2.waitKey(25) & 0xFF == ord('q'):
          break
    
    
    
    
cv2.destroyAllWindows()
        

True


error: OpenCV(4.5.3) :-1: error: (-5:Bad argument) in function 'flip'
> Overload resolution failed:
>  - src is not a numerical tuple
>  - Expected Ptr<cv::UMat> for argument 'src'


In [101]:
imu_1 = pd.read_csv(base_pth+"//imu01.csv")
# imu_2 = pd.read_csv(base_pth+"//2_imu.csv")

In [102]:
def datetime_to_s(df, c_name, sys_t=None, offset=0):
    """this function changes datetime string to seconds"""
    inf = datetime.strptime(df[c_name][0], '%Y-%m-%d %H:%M:%S.%f')
    dif_list = []
    for idx, val in enumerate(df[c_name]):
        _val = datetime.strptime(val, '%Y-%m-%d %H:%M:%S.%f')
        if sys_t:
            # diff = (_val - inf) - sys_t
            diff = _val - sys_t

        else:
            diff = _val - inf
            
        _diff = diff.total_seconds()
        # df["seconds"].loc[idx] = _diff
        dif_list.append(_diff)
    df["sys_sec"] = dif_list
    return df

In [103]:
# plt.plot(imu_1["imu_time"], imu_1["ax"])
imu_1 = datetime_to_s(imu_1, "sys_time", v_start)
# imu_2 = datetime_to_s(imu_2, "sys_time", v_start)

"""for df"""

imu_1df = pd.read_csv(base_pth+"//imu01.csv", parse_dates=['sys_time'], index_col='sys_time')
imu_1df.index.name = "time"
# imu_2df = pd.read_csv(base_pth+"//2_imu.csv", parse_dates=['sys_time'], index_col='sys_time')
# imu_2df.index.name = "time"
vid_df = pd.DataFrame(video_time_df, columns=["time"])
vid_df = pd.to_datetime(vid_df["time"])

# imu_1df = imu_1df.rename(columns={'ax.1': 'az'})
imu_1df

,gx,gy,gz,ax,ay,az
time,,,,,,
2022-02-18 13:03:36.060240,0.018289,0.059847,-0.030717,-0.059496,0.299166,1.051833
2022-02-18 13:03:36.196563,0.155694,0.014046,-0.015450,-0.056811,0.295016,1.050124
2022-02-18 13:03:36.197563,0.201495,-0.062290,-0.015450,-0.057543,0.291354,1.048415
2022-02-18 13:03:36.197563,0.094625,-0.031756,-0.030717,-0.054369,0.295016,1.052077
2022-02-18 13:03:36.197563,-0.012245,-0.062290,-0.000183,-0.053393,0.290865,1.045485
...,...,...,...,...,...,...
2022-02-18 13:04:02.661442,-0.149650,0.105649,0.030352,-0.059496,0.289400,1.048415
2022-02-18 13:04:02.661442,-0.073314,-0.062290,0.015085,-0.056322,0.291598,1.048903
2022-02-18 13:04:02.661442,-0.256520,0.014046,-0.015450,-0.056566,0.290133,1.048659


In [104]:
# imu_1df = imu_1df.rename(columns={'ax.1': 'az'})
# imu_1df
imu_1df.index

DatetimeIndex(['2022-02-18 13:03:36.060240', '2022-02-18 13:03:36.196563',
               '2022-02-18 13:03:36.197563', '2022-02-18 13:03:36.197563',
               '2022-02-18 13:03:36.197563', '2022-02-18 13:03:36.197563',
               '2022-02-18 13:03:36.197563', '2022-02-18 13:03:36.198563',
               '2022-02-18 13:03:36.198563', '2022-02-18 13:03:36.198563',
               ...
               '2022-02-18 13:04:02.630756', '2022-02-18 13:04:02.631758',
               '2022-02-18 13:04:02.631758', '2022-02-18 13:04:02.631758',
               '2022-02-18 13:04:02.631758', '2022-02-18 13:04:02.661442',
               '2022-02-18 13:04:02.661442', '2022-02-18 13:04:02.661442',
               '2022-02-18 13:04:02.662407', '2022-02-18 13:04:02.662407'],
              dtype='datetime64[ns]', name='time', length=4122, freq=None)

In [105]:
# plt.plot(imu_1["sys_sec"], imu_1["az"])
# imu_1df
vid_df

0     2022-02-18 13:03:29.286814
1     2022-02-18 13:03:29.354435
2     2022-02-18 13:03:29.420310
3     2022-02-18 13:03:29.487527
4     2022-02-18 13:03:29.555044
                 ...            
507   2022-02-18 13:04:03.085520
508   2022-02-18 13:04:03.153857
509   2022-02-18 13:04:03.219161
510   2022-02-18 13:04:03.285972
511   2022-02-18 13:04:03.353281
Name: time, Length: 512, dtype: datetime64[ns]

In [106]:
imutime = [] 
for t in tqdm(vid_df[:len(frames)-1]):
    
    # print(t)
    imutime.append(imu_1df.index.unique()[imu_1df.index.unique().get_loc(t, method='nearest')])
grps = []
for t in tqdm(imutime):
    grps.append(imu_1df.between_time((t - timedelta(seconds=1)).time(), (t + timedelta(seconds=1)).time()))

100%|██████████| 135/135 [00:00<00:00, 4994.50it/s]


In [107]:
imutime

[Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.060240'),
 Timestamp('2022-02-18 13:03:36.06

In [108]:
# imutime2 = [] 
# for t in tqdm(vid_df[:len(frames)-1]):
    
#     # print(t)
#     imutime2.append(imu_2df.index.unique()[imu_2df.index.unique().get_loc(t, method='nearest')])
# grps2 = []
# for t in tqdm(imutime2):
#     grps2.append(imu_2df.between_time((t - timedelta(seconds=2)).time(), (t + timedelta(seconds=1)).time()))

In [109]:
imu_1df.index.unique()[imu_1df.index.unique().get_loc(t, method='nearest')]

Timestamp('2022-02-18 13:03:38.222704')

In [110]:
# imu_1df.index.get_loc("2022-02-04 14:04:38.745642", method="nearest")

# imu_1df.index[0]
%matplotlib qt

In [111]:
grps

[                                  gx        gy        gz        ax        ay  \
 time                                                                           
 2022-02-18 13:03:36.060240  0.018289  0.059847 -0.030717 -0.059496  0.299166   
 2022-02-18 13:03:36.196563  0.155694  0.014046 -0.015450 -0.056811  0.295016   
 2022-02-18 13:03:36.197563  0.201495 -0.062290 -0.015450 -0.057543  0.291354   
 2022-02-18 13:03:36.197563  0.094625 -0.031756 -0.030717 -0.054369  0.295016   
 2022-02-18 13:03:36.197563 -0.012245 -0.062290 -0.000183 -0.053393  0.290865   
 ...                              ...       ...       ...       ...       ...   
 2022-02-18 13:03:37.047777  0.094625 -0.047023  0.121955 -0.050707  0.292086   
 2022-02-18 13:03:37.047777  0.018289 -0.245496 -0.076518 -0.058031  0.295504   
 2022-02-18 13:03:37.047777  0.003022  0.029313 -0.000183 -0.056811  0.295992   
 2022-02-18 13:03:37.047777  0.018289 -0.047023 -0.015450 -0.053881  0.289156   
 2022-02-18 13:03:37.047777 

In [112]:
start_at = 50
i=0

for frame, t, dat in zip(frames[start_at+1:], imutime[start_at:], grps[start_at:]):
    
    if keyboard.is_pressed('q'):
        plt.close()
        break
    else:
        plt.clf()
        plt.subplot(121)
        plt.title(i)
        plt.imshow(frame)
        plt.subplot(322)
        plt.title(t)
        plt.plot(dat.ax)
        plt.axvline(t, color='r')
        plt.ylim(-3, 3)
        plt.subplot(324)
        plt.plot(dat.ay)
        plt.axvline(t, color='r')
        plt.ylim(-3, 3)
        plt.subplot(326)
        plt.plot(dat.az)
        plt.axvline(t, color='r')
        plt.ylim(-3, 3)
        i+=1
        plt.pause(0.005)
    
    

TypeError: Invalid shape () for image data

In [ ]:
plt.plot(imutime2)

In [ ]:
plt.plot(imu_1df["az"])

In [ ]:
# plt.plot(t, dat.ax)

In [ ]:
plt.plot(imu_1df["imu_time"].diff())